<a href="https://colab.research.google.com/github/LennyBijan/MangioneDashboard/blob/main/Mangione_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports & pip

In [ ]:
!pip install plotly pandas ipywidgets pyngrok streamlit streamlit_option_menu

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import io
import numpy as np
import streamlit as st
import subprocess
from google.colab import files
from pyngrok import ngrok

## Excel hochladen

In [ ]:
uploaded = files.upload()

## Diagram & Metriken erstellen

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import plotly.graph_objects as go
from streamlit_option_menu import option_menu

# Define the Excel file path
uploaded_file = 'Mangione.xlsx'

# Load the Excel data
df = pd.read_excel(uploaded_file, header=9)
df['Rang'] = df['Final Score'].rank(ascending=False, method='min').astype(int)
visualization_data = df[['Applikationsname', 'Functional Fit', 'Technical Fit', 'Expiry', 'Risk / Complexity', 'Business Criticality', 'Rating', 'Wertschöpfung', 'Leistung', 'Final Score', 'Rang']]

# Define colors for ratings
rating_colors = {
    'A': 'green',
    'B': 'lime',
    'C': 'yellow',
    'D': 'orange',
    'E': 'red',
    'F': 'darkred'
}

# Function to display stars
def display_stars(value, total_stars=4):
    if pd.isna(value):
        return 'No data'
    filled_stars = '<span style="color: gold;">' + '★' * int(value) + '</span>'
    empty_stars = '<span style="color: gray;">' + '☆' * (total_stars - int(value)) + '</span>'
    return filled_stars + empty_stars

# Function to display rating
def display_rating(rating):
    color = rating_colors.get(rating, 'black')  # Default to black if rating is not in the dictionary
    return f'<span style="color: {color}; font-weight: bold; font-size: 24px;">{rating}</span>'

# Erste Linie von (1.3, 0) bis (3, 1.5)
# Berechnung der Steigung und des y-Abschnitts
slope1 = (1.5 - 0) / (3 - 1.3)
intercept1 = 0 - slope1 * 1.3
x_values1 = [1.3, 3]
y_values1 = [slope1 * x + intercept1 for x in x_values1]

# Zweite Linie von (0, 0.9) bis (2, 3)
# Berechnung der Steigung und des y-Abschnitts
slope2 = (3 - 0.9) / (2 - 0)
intercept2 = 0.9
x_values2 = [0, 2]
y_values2 = [slope2 * x + intercept2 for x in x_values2]

# Categorize function for plot data
def categorize_point(x, y):
    y_line1 = slope1 * x + intercept1
    y_line2 = slope2 * x + intercept2
    if y < y_line1:
        return 'Grün'  # Below line 1 and above line 2 (bottom right)
    elif y > y_line1 and y < y_line2:
        return 'Gelb'  # Below both lines (between the lines)
    elif y > y_line2:
        return 'Rot'  # Above line 1 (regardless of line 2, top left)

plot_data = df[['Applikationsname', 'Wertschöpfung', 'Leistung']]
df['Kategorie'] = df.apply(lambda row: categorize_point(row['Wertschöpfung'], row['Leistung']), axis=1)

def display_investment_decision(category):
    color_map = {'Grün': 'green', 'Gelb': 'yellow', 'Rot': 'red'}
    color = color_map.get(category, 'black')  # Default to black if category is not in the dictionary
    return f'<span style="color: {color}; font-size: 24px;">●</span>'

# Create placeholders for each type of content
metrics_placeholder = st.empty()
diagrams_placeholder = st.empty()
table_placeholder = st.empty()

# Create a tabbed interface
with st.sidebar:
    selected_tab = option_menu(
        menu_title="Main Menu",
        options=["Metrics", "Diagram", "Table"],
        icons=["bar-chart", "graph-up", "table"],
        menu_icon="cast",
        default_index=0,
    )

# Display content based on the selected tab
if selected_tab == "Metrics":
    with metrics_placeholder.container():
        st.title("Tool Metrics Visualization")
        tool = st.selectbox('Select a Tool', [''] + visualization_data['Applikationsname'].tolist())

        if tool:
            tool_data = visualization_data[visualization_data['Applikationsname'] == tool].iloc[0]
            category = df[df['Applikationsname'] == tool]['Kategorie'].values[0]

            st.markdown(f"**Rating:** {display_rating(tool_data['Rating'])}", unsafe_allow_html=True)
            st.markdown(f"**Functional Fit:** {display_stars(tool_data['Functional Fit'])}", unsafe_allow_html=True)
            st.markdown(f"**Technical Fit:** {display_stars(tool_data['Technical Fit'])}", unsafe_allow_html=True)
            st.markdown(f"**Expiry:** {display_stars(tool_data['Expiry'])}", unsafe_allow_html=True)
            st.markdown(f"**Risk / Complexity:** {display_stars(tool_data['Risk / Complexity'])}", unsafe_allow_html=True)
            st.markdown(f"**Business Criticality:** {display_stars(tool_data['Business Criticality'])}", unsafe_allow_html=True)
            st.markdown(f"**Investitionsentscheidung:** {display_investment_decision(category)}", unsafe_allow_html=True)
            st.markdown(f"**Rang:** Platz {tool_data['Rang']}", unsafe_allow_html=True)
    diagrams_placeholder.empty()  # Clear diagrams content when not in use

elif selected_tab == "Diagram":
    with diagrams_placeholder.container():
        st.title("Diagram of Wertschöpfungspotenzial and Leistungsfähigkeit")
        tool = st.selectbox('Select a Tool to display', ['All'] + visualization_data['Applikationsname'].tolist(), index=0)

        # Filter the data based on the selected tool
        if tool != 'All':
            filtered_data = df[df['Applikationsname'] == tool]
        else:
            filtered_data = df

        # Generate the diagram
        fig = go.Figure()

        # Define and add area traces for different investment decisions
        fig.add_trace(go.Scatter(
            x=[0, 1.3, 3, 3, 0],
            y=[0, slope1 * 1.3 + intercept1, slope1 * 3 + intercept1, 0, 0],
            fill='toself',
            fillcolor='rgba(0, 255, 0, 0.2)',
            line=dict(color='rgba(255,255,255,0)'),
            showlegend=False
        ))

        fig.add_trace(go.Scatter(
            x=[0, 0, 3, 3, 0],
            y=[0, 0.9, slope2 * 3 + intercept2, 0],
            fill='toself',
            fillcolor='rgba(218, 165, 32, 0.2)',
            line=dict(color='rgba(255,255,255,0)'),
            showlegend=False
        ))

        fig.add_trace(go.Scatter(
            x=[0, 3, 3, 0],
            y=[slope2 * 0 + intercept2, slope2 * 3 + intercept2, 3, 3],
            fill='toself',
            fillcolor='rgba(255, 0, 0, 0.2)',
            line=dict(color='rgba(255,255,255,0)'),
            showlegend=False
        ))

        # Add categorized points with hover templates
        for category, color, symbol, label in [
            ('Rot', 'red', 'triangle-up', 'nicht investieren'),
            ('Gelb', 'darkgoldenrod', 'square', 'bedingt investieren'),
            ('Grün', 'green', 'circle', 'investieren')]:
            cat_data = filtered_data[filtered_data['Kategorie'] == category]
            fig.add_trace(go.Scatter(
                x=cat_data['Wertschöpfung'],
                y=cat_data['Leistung'],
                mode='markers',
                marker=dict(symbol=symbol, size=10, color=color),
                name=f"{label}",
                text=cat_data['Applikationsname'],
                hovertemplate=(
                    "Applikation: %{text}<br>"
                    "Leistung: %{y}<br>"
                    "Wertschöpfung: %{x}<br>"
                    f"Investitionsentscheidung: <span style='color: {color};'>●</span><br>"
                    "Rating: %{customdata[0]}<extra></extra>"
                ),
                customdata=cat_data.apply(lambda row: [display_rating(row['Rating'])], axis=1),
                showlegend=False,
                hoverlabel=dict(
                    namelength=-1,
                    align='left'
                )
            ))

        # Add lines for visual separation
        fig.add_trace(go.Scatter(
            x=[1.3, 3],
            y=[slope1 * 1.3 + intercept1, slope1 * 3 + intercept1],
            mode='lines',
            name='Linie 1',
            line=dict(color='orange')
        ))

        fig.add_trace(go.Scatter(
            x=[0, 3],
            y=[slope2 * 0 + intercept2, slope2 * 3 + intercept2],
            mode='lines',
            name='Linie 2',
            line=dict(color='blue')
        ))

        # Customize axes and layout
        fig.update_xaxes(range=[-0.25, 3.25])
        fig.update_yaxes(range=[-0.25, 3])
        fig.update_layout(
            autosize=False,
            width=800,
            height=600,
            title='Interaktives Diagramm von Wertschöpfungspotenzial und Leistungsfähigkeit',
            legend=dict(
                x=1,
                y=1,
                title_text='Bereich',
                orientation='v'
            )
        )

        st.plotly_chart(fig)
        # New: Display a table of tools divided by investment decisions into three columns
        st.subheader("Investment Decision Classification")
        col1, col2, col3 = st.columns(3)

        # Filter data for each category
        data_rot = df[df['Kategorie'] == 'Rot']['Applikationsname'].tolist()
        data_gelb = df[df['Kategorie'] == 'Gelb']['Applikationsname'].tolist()
        data_grün = df[df['Kategorie'] == 'Grün']['Applikationsname'].tolist()

        with col1:
            st.markdown("### Rot (nicht investieren)")
            st.write(data_rot)

        with col2:
            st.markdown("### Gelb (bedingt investieren)")
            st.write(data_gelb)

        with col3:
            st.markdown("### Grün (investieren)")
            st.write(data_grün)


    metrics_placeholder.empty()  # Clear metrics content when not in use

elif selected_tab == "Table":
    with table_placeholder.container():
        st.title("Applications Table")

        # Add a text input for filtering the table
        search = st.text_input("Search for an application")

        # Filter the dataframe based on the search input
        filtered_data = visualization_data[visualization_data['Applikationsname'].str.contains(search, case=False, na=False)]

        # Display the filtered dataframe
        st.dataframe(filtered_data)

    metrics_placeholder.empty()  # Clear metrics content when not in use


if __name__ == "__main__":
    import subprocess
    subprocess.run(["streamlit", "run", "app.py"])

## Web Interface starten

In [ ]:
# Ensure your ngrok authtoken is set (replace with your actual token if necessary)
ngrok.set_auth_token("your-auth-token")

# Open a tunnel on port 8501 (default Streamlit port)
public_url = ngrok.connect(8501)
print(f"Public URL: {public_url}")

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501